In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import numba as nb
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from numbalsoda import lsoda_sig, lsoda
from numba import types
import scienceplots
import os

# Using seaborn's style
plt.style.use('science')
width = 'thesis'

def set_size(width, fraction=1, subplots=(1, 1)):
    """Set figure dimensions to avoid scaling in LaTeX.

    Parameters
    ----------
    width: float or string
            Document width in points, or string of predined document type
    fraction: float, optional
            Fraction of the width which you wish the figure to occupy
    subplots: array-like, optional
            The number of rows and columns of subplots.
    Returns
    -------
    fig_dim: tuple
            Dimensions of figure in inches
    """
    if width == 'thesis':
        width_pt = 426.79135
    elif width == 'beamer':
        width_pt = 307.28987
    else:
        width_pt = width

    # Width of figure (in pts)
    fig_width_pt = width_pt * fraction
    # Convert from pt to inches
    inches_per_pt = 1 / 72.27

    # Golden ratio to set aesthetic figure height
    # https://disq.us/p/2940ij3
    golden_ratio = (5**.5 - 1) / 2

    # Figure width in inches
    fig_width_in = fig_width_pt * inches_per_pt
    # Figure height in inches
    fig_height_in = fig_width_in * golden_ratio * (subplots[0] / subplots[1])

    return (fig_width_in, fig_height_in)

In [ ]:
@nb.njit(fastmath=True)
def promoterActivity(A, R, K_TF_GENE, K_REP_GENE, K_1, K_2, c, n_TF, n_REP):

    return ((K_TF_GENE * K_2 * c * ((K_1 * A) / (1 + K_1 * A)) ** n_TF)) / (
        1 + ((K_TF_GENE * K_2 * c * ((K_1 * A) / (1 + K_1 * A)) ** n_TF)) + (K_REP_GENE*R)**n_REP)

@nb.cfunc(lsoda_sig)
def ferroSimplified(t, u, du, p):
    
    # Equations in order represent the following:
    # 0 = C6i
    # 1 = C12i
    # 2 = C6e
    # 3 = C12e
    # 4 = TetR repressor
    # 5 = LacI repressor
    # 6 = LuxI enzyme
    # 7 = LasI enzyme
    # 8 = RFP (mCherry)
    # 9 = GFP (sfGFP)    

    du[0] = (p[4]*u[6])/(10.0 + u[6]) + p[6]*(u[2] - u[0]) - p[7]*u[0]
    
    du[1] = (p[5]*u[7])/(10.0 + u[7]) + p[6]*(u[3] - u[1]) - p[7]*u[1]
    
    du[2] = p[6]*(u[0] - u[2]) - p[8]*u[2]
    
    du[3] = p[6]*(u[1] - u[3]) - p[8]*u[3]

    du[4] = (promoterActivity(u[1], u[5], p[10], p[12], p[18], p[22], p[23], p[14], p[20]) * p[0] + p[16]) * (p[1]/p[2]) - p[3]*u[4]

    du[5] = (promoterActivity(u[0], u[4], p[9], p[11], p[17], p[21], p[23], p[13], p[19]) * p[0] + p[15]) * (p[1]/p[2]) - p[3]*u[5]
    
    du[6] = (promoterActivity(u[0], u[4], p[9], p[11], p[17], p[21], p[23], p[13], p[19]) * p[0] + p[15]) * (p[1]/p[2]) - p[3]*u[6]

    du[7] = (promoterActivity(u[1], u[5], p[10], p[12], p[18], p[22], p[23], p[14], p[20]) * p[0] + p[16]) * (p[1]/p[2]) - p[3]*u[7]

    du[8] = (promoterActivity(u[0], u[4], p[9], p[11], p[17], p[21], p[23], p[13], p[19]) * p[0] + p[15]) * (p[1]/p[2]) - p[3]*u[8]
    
    du[9] = (promoterActivity(u[1], u[5], p[10], p[12], p[18], p[22], p[23], p[14], p[20]) * p[0] + p[16]) * (p[1]/p[2]) - p[3]*u[9]

In [ ]:
# Define ranges for the external C6 concentrations
one = np.linspace(1e-6, 1e-5, 100)
two = np.linspace(1e-5, 1e-4, 100)
three = np.linspace(1e-4, 1e-3, 100)
four = np.linspace(1e-3, 1e-2, 100)
five = np.linspace(1e-2, 1e-1, 100)
six = np.linspace(1e-1, 1, 100)

# Join the ranges together
conc = np.concatenate((one, two, three, four, five, six))
# And remove the repeated values
conc = np.unique(conc)

# Create a meshgrid of the concentrations
conc1, conc2 = np.meshgrid(conc, conc)

# Define the different membrane diffusion coefficients
membrane_coefficients = np.array([
        0.01,
        0.05,
        0.1,
        0.25,
        0.5,
        0.75,
        1.0,
        3.0,
        5.0,
        10.0,
    ])
n = len(membrane_coefficients)

In [ ]:
funcptr_simplified = ferroSimplified.address
eval_time = np.linspace(0, 1440, 1440*10)

u0_simplified = np.array([
    0.0,                # Initial condition number 0
    0.0,                # Initial condition number 1
    0.0,                # Initial condition number 2
    0.0,                # Initial condition number 3
    0.0,                # Initial condition number 4
    0.0,                # Initial condition number 5
    0.0,                # Initial condition number 6
    0.0,                # Initial condition number 7
    0.0,                # Initial condition number 8
    0.0,                # Initial condition number 9
])

params_simplified = np.array([
    1e-1, # k_tx # p[0]
    5e0, # k_tl # p[1]
    5e-1, # d_m # p[2]
    5e-2, # d_p # p[3]
    0.04, # KLUXI # p[4]
    0.04, # KLASI # p[5]
    0.01, # membrane transport rate of AHL # p[6]
    0.01, # d_HSLi # p[7]
    0.001, # d_HSLe # p[8]
    1/0.10, # K_GR # p[9]
    1/0.10, # K_GS # p[10]
    1/0.001, # K_TET # p[11] 
    1/0.100, # K_LAC # p[12]
    2.0, # n_R # p[13] 
    2.0, # n_S # p[14]
    5e-5,  # beta_0 # p[15]
    2.5e-4,  # beta_1 # p[16]
    1/0.1, # K_1R # p[17]
    1/0.1, # K_1S # p[18]
    2.0, # n_REP1 # p[19]
    4.0, # n_REP2 # p[20]
    1/0.02, # K_2R # p[21]
    1/0.02, # K_2S # p[22]
    ((1e-1*5e0)/(5e-1*5e-2))**2 # c # p[23]
])  

# Create an empty array to store the results
results = np.zeros((len(conc), 2, n))

@nb.njit(fastmath=True)
def main_loop_simplified(func, conc, membrane_coefficients, params, u0, eval_time, n, out=results):
    # Loop over the membrane coefficients
    for i in range(n):
        params[6] = membrane_coefficients[i]
        # Loop over the concentrations
        for j in range(len(conc)):

            # Change the external concentrations
            u0[2] = conc[j]

            # Solve the system
            usol, _ = lsoda(func, u0, eval_time, data = params)

            # Store only the last point for the 15th and 16th variables
            out[j, :, i] = usol[-1, 8:]

    return out, usol

resultsSimplified, solutionsSimplified = main_loop_simplified(funcptr_simplified, conc, membrane_coefficients, params_simplified, u0_simplified, eval_time, n, out=results)

# 1. Simulating the ODE system for the ferromagnetic cell

## 1.1 LuxI and LasI Dynamics

In [ ]:
# Simulate the system for an external C6 concentration of 0 and 1, plot the graphs one on top of the other
# and save the figure as a pdf file with 300 dpi
low_C6 = np.array([
    0.0,                # Initial condition number 0
    0.0,                # Initial condition number 1
    0.0,                # Initial condition number 2
    0.0,                # Initial condition number 3
    0.0,                # Initial condition number 4
    0.0,                # Initial condition number 5
    0.0,                # Initial condition number 6
    0.0,                # Initial condition number 7
    0.0,                # Initial condition number 8
    0.0,                # Initial condition number 9
])
high_C6 = np.array([
    0.0,                # Initial condition number 0
    0.0,                # Initial condition number 1
    0.0,                # Initial condition number 2
    0.0,                # Initial condition number 3
    0.0,                # Initial condition number 4
    0.0,                # Initial condition number 5
    0.0,                # Initial condition number 6
    0.0,                # Initial condition number 7
    0.0,                # Initial condition number 8
    0.0,                # Initial condition number 9
])

low_C6[2] = 0.0
high_C6[2] = 1.0

low_C6_sol, _ = lsoda(funcptr_simplified, low_C6, eval_time, data = params_simplified)
high_C6_sol, _ = lsoda(funcptr_simplified, high_C6, eval_time, data = params_simplified)

# Plot using matplotlib
fig, ax = plt.subplots(2, 1, sharex=True, figsize=set_size(width, fraction=1, subplots=(2, 1)))

ax[0].plot(eval_time, low_C6_sol[:, 6], label='LuxI', color='tab:red', lw=5)
ax[0].plot(eval_time, low_C6_sol[:, 7], label='LasI', color='tab:green', lw=5)
ax[0].set_ylabel('Concentración ($\mu$M)')
# Legend at the center right
ax[0].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 0
ax[0].set_title('C6-AHL externo = 0 $\mu$M')

ax[1].plot(eval_time, high_C6_sol[:, 6], label='LuxI', color='tab:red', lw=5)
ax[1].plot(eval_time, high_C6_sol[:, 7], label='LasI', color='tab:green', lw=5)
ax[1].set_ylabel('Concentración ($\mu$M)')
ax[1].set_xlabel('Tiempo (min)')
# Legend at the center right
ax[1].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 1
ax[1].set_title('C6-AHL externo = 1 $\mu$M')

# Put a superior title
fig.suptitle('Concentración de LuxI y LasI en función del tiempo\na diferentes concentraciones de C6-AHL externo')

plt.tight_layout()
plt.savefig('THESIS_FIGURES/LUXI_LASI_COMPARACION.pdf', dpi=300)
plt.close()

## 1.2 TetR and LacI Dynamics

In [ ]:
# Do the same for the LuxI and LasI dynamics but for TetR and LacI
# Plot using matplotlib
fig, ax = plt.subplots(2, 1, sharex=True, figsize=set_size(width, fraction=1, subplots=(2, 1)))

ax[0].plot(eval_time, low_C6_sol[:, 5], label='LacI', color='tab:red', lw=5)
ax[0].plot(eval_time, low_C6_sol[:, 4], label='TetR', color='tab:green', lw=5)
ax[0].set_ylabel('Concentración ($\mu$M)')
# Legend at the center right
ax[0].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 0
ax[0].set_title('C6-AHL externo = 0 $\mu$M')

ax[1].plot(eval_time, high_C6_sol[:, 5], label='LacI', color='tab:red', lw=5)
ax[1].plot(eval_time, high_C6_sol[:, 4], label='TetR', color='tab:green', lw=5)
ax[1].set_ylabel('Concentración ($\mu$M)')
ax[1].set_xlabel('Tiempo (min)')
# Legend at the center right
ax[1].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 1
ax[1].set_title('C6-AHL externo = 1 $\mu$M')

# Put a superior title
fig.suptitle('Concentración de LacI y TetR en función del tiempo\na diferentes concentraciones de C6-AHL externo')

plt.tight_layout()
plt.savefig('THESIS_FIGURES/TETR_LACI_COMPARACION.pdf', dpi=300)
plt.close()

## 1.3 C6 and C12 intracellular and extracellular dynamics


In [ ]:
# Do the same for the LuxI and LasI dynamics but for C6i, C12i, C6e and C12e
# Plot using matplotlib
fig, ax = plt.subplots(2, 1, sharex=True, figsize=set_size(width, fraction=1, subplots=(2, 1)))

ax[0].plot(eval_time, low_C6_sol[:, 0], label='C6i', color='tab:red', lw=5)
ax[0].plot(eval_time, low_C6_sol[:, 1], label='C12i', color='tab:green', lw=5)
ax[0].plot(eval_time, low_C6_sol[:, 2], label='C6e', color='tab:red', lw=5, linestyle='dashed')
ax[0].plot(eval_time, low_C6_sol[:, 3], label='C12e', color='tab:green', lw=5, linestyle='dashed')
ax[0].set_ylabel('Concentración ($\mu$M)')
# Legend at the center right
ax[0].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 0
ax[0].set_title('C6-AHL externo = 0 $\mu$M')

ax[1].plot(eval_time, high_C6_sol[:, 0], label='C6i', color='tab:red', lw=5)
ax[1].plot(eval_time, high_C6_sol[:, 1], label='C12i', color='tab:green', lw=5)
ax[1].plot(eval_time, high_C6_sol[:, 2], label='C6e', color='tab:red', lw=5, linestyle='dashed')
ax[1].plot(eval_time, high_C6_sol[:, 3], label='C12e', color='tab:green', lw=5, linestyle='dashed')
ax[1].set_ylabel('Concentración ($\mu$M)')
ax[1].set_xlabel('Tiempo (min)')
# Legend at the center right
ax[1].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 1
ax[1].set_title('C6-AHL externo = 1 $\mu$M')

# Put a superior title
fig.suptitle('Concentración de C6i, C12i, C6e y C12e en función del tiempo\na diferentes concentraciones de C6-AHL externo')

plt.tight_layout()
plt.savefig('THESIS_FIGURES/C6_C12_COMPARACION.pdf', dpi=300)
plt.close()

## 1.4 mCherry2 (RFP) and sfGFP (GFP) dynamics

In [ ]:
# Do the same for the LuxI and LasI dynamics but for GFP and RFP
# Plot using matplotlib
fig, ax = plt.subplots(2, 1, sharex=True, figsize=set_size(width, fraction=1, subplots=(2, 1)))

ax[0].plot(eval_time, low_C6_sol[:, 8], label='mCherry2', color='tab:red', lw=5)
ax[0].plot(eval_time, low_C6_sol[:, 9], label='sfGFP', color='tab:green', lw=5)
ax[0].set_ylabel('Concentración ($\mu$M)')
# Legend at the center right
ax[0].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 0
ax[0].set_title('C6-AHL externo = 0 $\mu$M')

ax[1].plot(eval_time, high_C6_sol[:, 8], label='mCherry2', color='tab:red', lw=5)
ax[1].plot(eval_time, high_C6_sol[:, 9], label='sfGFP', color='tab:green', lw=5)
ax[1].set_ylabel('Concentración ($\mu$M)')
ax[1].set_xlabel('Tiempo (min)')
# Legend at the center right
ax[1].legend(loc='center right')
# Add a title for the subplot with the external C6 concentration of 1
ax[1].set_title('C6-AHL externo = 1 $\mu$M')

# Put a superior title
fig.suptitle('Concentración de mCherry2 y sfGFP en función del tiempo\na diferentes concentraciones de C6-AHL externo')

plt.tight_layout()
plt.savefig('THESIS_FIGURES/RFP_GFP_COMPARACION.pdf', dpi=300)
plt.close()

# Many concentrations for the switching point

In [ ]:
def solutionFunc(func, conc, params, u0, eval_time, ax0=None, ax1=None, membraneCoeff=None, colormap=None):

    params[6] = membraneCoeff
    # Loop over the concentrations
    for j in range(len(conc)):

        # Change the external concentrations
        u0[2] = conc[j]

        # Solve the system
        usol, results = lsoda(func, u0, eval_time, data = params)

        # Plot the mCherry2 and sfGFP dynamics
        ax0.plot(eval_time, usol[:, 8], color=colormap[j], lw=5)
        ax1.plot(eval_time, usol[:, 9], color=colormap[j], lw=5)

In [ ]:
# Create a colormap from green to red, with a length of the concentration array
cmap = plt.get_cmap('viridis')
colors = cmap(np.linspace(0.025, 0.975, len(conc)))
# Create a figure with 2 subplots, one on top of the other
fig, ax = plt.subplots(2, 1, sharex=True, figsize=set_size(width, fraction=1, subplots=(2, 1)))

# Plot the mCherry2 (RFP) and sfGFP concentrations for all concentrations of C6-AHL used
# Choose a membrane diffusion coefficient of 1.0
solutionFunc(funcptr_simplified, conc, params_simplified, u0_simplified, eval_time, ax0=ax[0], ax1=ax[1], membraneCoeff=1.0, colormap=colors)

# Change the y axis scale to logarithmic
ax[0].set_yscale('log')
ax[1].set_yscale('log')

# Change the limit of the x axis to 360 mins
ax[0].set_xlim(0, 360)
ax[1].set_xlim(0, 360)

# Set the names of the axis
ax[0].set_ylabel('Concentración ($\mu$M)')
ax[1].set_ylabel('Concentración ($\mu$M)')
ax[1].set_xlabel('Tiempo (min)')

# Set a title for the subplots
ax[0].set_title('mCherry2')
ax[1].set_title('sfGFP')

# Set a superior title
fig.suptitle('Concentración de mCherry2 y sfGFP en función del tiempo\na diferentes concentraciones de C6-AHL externo')

plt.tight_layout()
plt.savefig('THESIS_FIGURES/RFP_GFP_PUNTO_DE_CAMBIO.pdf', dpi=300)
plt.close()

# Búsqueda del punto de cambio del sistema ferromagnetico

In [ ]:
criticalPointsSimplified = {}

# Loop over the membrane coefficients and the corresponding results
for i, j in zip(membrane_coefficients, resultsSimplified.T):

    # Find the index for the current results where the 15th and 16th variables are equal
    try:
        # Find the index where RFP = GFP
        idx = np.argwhere(np.diff(np.sign(j[0, :] - j[1, :]))).flatten()[0]
    except:
        # Return the lowest index where RFP > GFP
        idx = np.argwhere(j[0, :] > j[1, :]).flatten()

    # Store the critical point
    criticalPointsSimplified[i] = conc[idx]

# Create a red color map for C6 and a green color map for C12
cmap1 = plt.get_cmap("Reds", n)
cmap2 = plt.get_cmap("Greens", n)

# Make a list of n colors from the cmap1 colormap
red_colors = [tuple((np.array(cmap1(i)[:-1])*255).astype(int)) for i in range(n)]
# Now make them a string starting with rgb and then the values
red_colors = [f"rgb({i[0]}, {i[1]}, {i[2]})" for i in red_colors]

# Make a list of n colors from the cmap2 colormap
green_colors = [tuple((np.array(cmap2(i)[:-1])*255).astype(int)) for i in range(n)]
# Now make them a string starting with rgb and then the values
green_colors = [f"rgb({i[0]}, {i[1]}, {i[2]})" for i in green_colors]

# Create the same figure as the code cell above but with plotly
fig = make_subplots(rows=1, cols=1)

# Loop over the membrane coefficients and the corresponding results
# Create a sequential colormap usinng plotly for the number of membrane coefficients
counter = 0
for i, j in zip(membrane_coefficients, resultsSimplified.T):
    try:
        # Plot the results
        fig.add_trace(go.Scatter(x=conc, y=j[0, :], mode='lines', name=f'RFP; Dm: {i}', line=dict(color=red_colors[counter])))
        fig.add_trace(go.Scatter(x=conc, y=j[1, :], mode='lines', name=f'GFP; Dm: {i}, Crit C6: {round(criticalPointsSimplified[i], 6)}', line=dict(color=green_colors[counter])))
        counter += 1
    except:
        counter += 1

# Set the x axis to log scale
fig.update_xaxes(type="log", row=1, col=1)
fig.update_layout(height=500, 
                  width=1000,
                  title_text="Final RFP and GFP concentrations of the ODE model")
fig.show()

In [ ]:
# Do the same plot as above but with matplotlib
fig, ax = plt.subplots(1, 1, figsize=set_size(width, fraction=1, subplots=(1, 1)))

# Create a red color map for C6 and a green color map for C12
cmap1 = plt.get_cmap("Reds")
cmap2 = plt.get_cmap("Greens")
colors1 = cmap1(np.linspace(0.1, 0.9, n))
colors2 = cmap2(np.linspace(0.1, 0.9, n))
color_counter = 0

# Loop over the membrane coefficients and the corresponding results
for i, j in zip(membrane_coefficients, resultsSimplified.T):
    # Plot the results
    ax.plot(conc, j[0, :], color=colors1[color_counter])
    ax.plot(conc, j[1, :], color=colors2[color_counter])
    color_counter += 1

# Set the x axis to log scale
ax.set_xscale('log')

# Set the names of the axis
ax.set_ylabel('Concentración ($\mu$M)')
ax.set_xlabel('Concentración de C6-AHL externo ($\mu$M)')
# Set the limit of the x axis
ax.set_xlim(1e-6, 1)

# Set a title for the plot
ax.set_title('[mCherry2] y [sfGFP] en $t=t_f$ en función de C6-AHL externo\ny el coeficiente de permeabilidad de membrana ($D_m$)')

# Set a colorbar for the plot
sm = plt.cm.ScalarMappable(cmap=cmap1, norm=plt.Normalize(vmin=0, vmax=1))
sm._A = []
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.05, pad=0.05, ticks=membrane_coefficients)
cbar.set_label('$D_m$')

# Set a legend for the last two lines of the plot
ax.legend(['mCherry2', 'sfGFP'], loc='center right')

plt.tight_layout()
plt.savefig('THESIS_FIGURES/RFP_GFP_PUNTO_DE_CAMBIO_MATPLOTLIB.pdf', dpi=300)

In [ ]:
criticalPointsSimplified

In [ ]:
# For each Dm print the critical concentration
for i in criticalPointsSimplified:
    print(round(criticalPointsSimplified[i], 8))

In [ ]:
# Make the same graph for finding the critical points but with matplotlib
fig, ax = plt.subplots(figsize=set_size(width, fraction=0.75), dpi=300)
ax.set_xlabel('Tiempo (min)')
ax.set_ylabel('Concentración ($\mu M$)')
ax.set_xscale('log')

# Loop over the membrane coefficients and the corresponding results
for i, j in zip(membrane_coefficients, resultsSimplified.T):
    if i == 0.01:
        try:
            # Plot the results
            ax.plot(conc, j[0, :], label='mCherry2', lw=1.0, color='tab:red')
            ax.plot(conc, j[1, :], label='sfGFP', lw=1.0, color='tab:green')
        except:
            pass
    else:
        pass
ax.legend()
plt.tight_layout()
plt.savefig('THESIS_FIGURES/SWITCHING_CONCENTRATION.pdf', bbox_inches='tight')